<a href="https://colab.research.google.com/github/Vakhranev/Pushkina/blob/main/TFIDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# Установка соединения с Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
gc = gspread.authorize(credentials)

# Открытие таблицы по ID и выбор нужного листа
spreadsheet_id = '1aTMzHwwhrJ_b9DRId5gSsuAfVjTAuAC2u0hMsJWK9EI'
sheet_name = 'Sheet2'
worksheet = gc.open_by_key(spreadsheet_id).worksheet(sheet_name)

# Считывание данных с листа
data = worksheet.get_all_values()

# Преобразование данных в DataFrame
df = pd.DataFrame(data[1:], columns=data[0])

# Сохранение DataFrame в CSV файл
file_name = "лексика мигранты однословные (определение типа лексемы).csv"
df.to_csv(file_name, index=False)

print(f"Файл сохранён как {file_name}")

Файл сохранён как лексика мигранты однословные (определение типа лексемы).csv


In [2]:
!pip install PyDrive

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Аутентификация и создание клиента PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# ID папки на Google Диске
folder_id = '122VoUkLJSOp3uSKMQ5DevUq_q3bRlNvm'

# Получение списка файлов в папке
file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()

# Загрузка каждого файла
for file in file_list:
    print(f'Downloading {file["title"]} from Google Drive')
    file.GetContentFile(file['title'])

In [5]:
!pip install nltk spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
!pip install scikit-learn
!pip install gspread oauth2client

In [7]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=9119263345d76dc626adfb2b1a8fd6e851c67a3aef7de809a455ccfe0d15592f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [19]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pymorphy2
from collections import defaultdict

# Инициализация Pymorphy
morph = pymorphy2.MorphAnalyzer()

# Словарь для перевода тегов Pymorphy2 в теги таблицы
tag_translation = {
    'ADJF': 'прилагательное',
    'ADJS': 'прилагательное',
    'ADVB': 'наречие',
    'COMP': 'сравнительное',
    'CONJ': 'союз',
    'GRND': 'деепричастие',
    'INFN': 'глагол',
    'INTJ': 'междометие',
    'NOUN': 'существительное',
    'NPRO': 'местоимение-существительное',
    'NUMR': 'числительное',
    'PRCL': 'частица',
    'PRED': 'предикатив',
    'PREP': 'предлог',
    'PRTF': 'причастие',
    'PRTS': 'причастие',
    'VERB': 'глагол'
}

# Функция предобработки текста для Pymorphy
def pymorphy_preprocess(text):
    tokens = re.findall(r'\b\w+\b', text.lower())  # Токенизация с учетом только слов
    lemmas = []
    for token in tokens:
        parsed = morph.parse(token)[0]
        if parsed.tag.POS and parsed.tag.POS in tag_translation:
            lemmas.append(parsed.normal_form + '__' + tag_translation[parsed.tag.POS])
    return lemmas

# Загрузка данных из локального CSV файла
file_path = "лексика мигранты однословные (определение типа лексемы).csv"
df = pd.read_csv(file_path)

# Проверка наличия колонки 'lemma__pos'
if 'lemma__pos' not in df.columns:
    raise ValueError("Колонка 'lemma__pos' отсутствует в данных CSV")

# Приведение тегов в таблице к нижнему регистру
df['lemma__pos'] = df['lemma__pos'].str.replace(r'__([A-ZА-Я]+)', lambda x: '__' + x.group(1).lower(), regex=True)

lemmas_list = df['lemma__pos'].tolist()

# Проверка формата лемм
print("Пример лемм из таблицы:", lemmas_list[:10])

# Считывание файлов из Деткорпуса
corpus_dir = '/content/'  # Замените на ваш путь к директории с текстами
corpus_texts = []
for filename in os.listdir(corpus_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(corpus_dir, filename), 'r', encoding='utf-8') as file:
            corpus_texts.append(file.read())

# Предобработка текста Деткорпуса
processed_corpus_texts = [" ".join(pymorphy_preprocess(text)) for text in corpus_texts]

# Проверка предобработанного текста корпуса
print("Пример предобработанного текста корпуса:", processed_corpus_texts[:2])

# Объединение всех текстов корпуса в один текст для подсчета TF-IDF
corpus_text = " ".join(processed_corpus_texts)

# Подсчет TF-IDF для лемм из таблицы на основе корпуса
vectorizer = TfidfVectorizer(vocabulary=lemmas_list)
tfidf_matrix = vectorizer.fit_transform([corpus_text])

# Получение TF-IDF значений для слов из таблицы
tfidf_values = tfidf_matrix.toarray()[0]

# Обновление значений в DataFrame
df['tfidf'] = [tfidf_values[lemmas_list.index(lemma)] if lemma in lemmas_list else 0 for lemma in lemmas_list]

# Сохранение обновленного DataFrame в CSV файл
output_file_path = "обновленная_лексика_мигранты.csv"
df.to_csv(output_file_path, index=False)

print(f"Значения TF-IDF обновлены и сохранены в '{output_file_path}'.")

Пример лемм из таблицы: ['и__союз', 'в__предлог', 'на__предлог', 'с__предлог', 'а__союз', 'не__частица', 'быть__глагол', 'он__местоимение-существительное', 'к__предлог', 'что__союз']


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Значения TF-IDF обновлены и сохранены в 'обновленная_лексика_мигранты.csv'.


In [20]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pymorphy2

# Инициализация Pymorphy
morph = pymorphy2.MorphAnalyzer()

# Словарь для перевода тегов Pymorphy2 в теги таблицы
tag_translation = {
    'ADJF': 'прилагательное',
    'ADJS': 'прилагательное',
    'ADVB': 'наречие',
    'COMP': 'сравнительное',
    'CONJ': 'союз',
    'GRND': 'деепричастие',
    'INFN': 'глагол',
    'INTJ': 'междометие',
    'NOUN': 'существительное',
    'NPRO': 'местоимение-существительное',
    'NUMR': 'числительное',
    'PRCL': 'частица',
    'PRED': 'предикатив',
    'PREP': 'предлог',
    'PRTF': 'причастие',
    'PRTS': 'причастие',
    'VERB': 'глагол'
}

# Функция предобработки леммы для Pymorphy
def pymorphy_preprocess_lemma(lemma_pos):
    lemma, pos = lemma_pos.split('__')
    parsed = morph.parse(lemma)[0]
    if parsed.tag.POS and parsed.tag.POS in tag_translation:
        return parsed.normal_form + '__' + tag_translation[parsed.tag.POS]
    return lemma_pos

# Загрузка данных из CSV файла
file_path = "обновленная_лексика_мигранты.csv"
df = pd.read_csv(file_path)

# Проверка наличия колонок 'lemma__pos' и 'tfidf'
if 'lemma__pos' not in df.columns or 'tfidf' not in df.columns:
    raise ValueError("Колонки 'lemma__pos' или 'tfidf' отсутствуют в данных CSV")

# Обработка лемм для тех слов, у которых TF-IDF равен 0
df['lemma__pos_pymorphy'] = df.apply(
    lambda row: pymorphy_preprocess_lemma(row['lemma__pos']) if row['tfidf'] == 0 else row['lemma__pos'],
    axis=1
)

# Сохранение обновленного DataFrame в CSV файл
output_file_path = "обновленная_лексика_мигранты_с_леммами.csv"
df.to_csv(output_file_path, index=False)

print(f"Обновленный файл сохранен как '{output_file_path}'.")

Обновленный файл сохранен как 'обновленная_лексика_мигранты_с_леммами.csv'.


In [21]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pymorphy2

# Инициализация Pymorphy
morph = pymorphy2.MorphAnalyzer()

# Функция предобработки текста для Pymorphy
def pymorphy_preprocess(text):
    tokens = re.findall(r'\b\w+\b', text.lower())  # Токенизация с учетом только слов
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    return lemmas

# Загрузка данных из CSV файла
file_path = "обновленная_лексика_мигранты.csv"
df = pd.read_csv(file_path)

# Проверка наличия колонок 'lemma__pos' и 'tfidf'
if 'lemma__pos' not in df.columns or 'tfidf' not in df.columns:
    raise ValueError("Колонки 'lemma__pos' или 'tfidf' отсутствуют в данных CSV")

# Удаление части речи из леммы
df['lemma'] = df['lemma__pos'].apply(lambda x: x.split('__')[0])

# Считывание файлов из Деткорпуса
corpus_dir = '/content/'  # Замените на ваш путь к директории с текстами
corpus_texts = []
for filename in os.listdir(corpus_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(corpus_dir, filename), 'r', encoding='utf-8') as file:
            corpus_texts.append(file.read())

# Предобработка текста Деткорпуса
processed_corpus_texts = [" ".join(pymorphy_preprocess(text)) for text in corpus_texts]

# Проверка предобработанного текста корпуса
print("Пример предобработанного текста корпуса:", processed_corpus_texts[:2])

# Объединение всех текстов корпуса в один текст для подсчета TF-IDF
corpus_text = " ".join(processed_corpus_texts)

# Подсчет TF-IDF для лемм без учета части речи
unique_lemmas = df['lemma'].unique().tolist()
vectorizer = TfidfVectorizer(vocabulary=unique_lemmas)
tfidf_matrix = vectorizer.fit_transform([corpus_text])

# Получение TF-IDF значений для слов из таблицы
tfidf_values = tfidf_matrix.toarray()[0]
tfidf_dict = dict(zip(vectorizer.get_feature_names_out(), tfidf_values))

# Обновление значений TF-IDF для лемм с нулевым исходным TF-IDF
df['tfidf_updated'] = df.apply(lambda row: tfidf_dict[row['lemma']] if row['tfidf'] == 0 else row['tfidf'], axis=1)

# Сохранение обновленного DataFrame в CSV файл
output_file_path = "обновленная_лексика_мигранты_с_обновленным_tfidf.csv"
df.to_csv(output_file_path, index=False)

print(f"Обновленный файл сохранен как '{output_file_path}'.")

Пример предобработанного текста корпуса: ['просвещение издательство ряо57 йу учебник для общеобразовательный организация под редакция академик ран а в торкунов в два часть часть 1 рекомендовать министерство образование и наука российский федерация москва просвещение 2016 удк 373 167 1 94 47 ббк 63 3 2 я72 и90 автор н м арсентьев а а данилов п с стефанович а я токарев методический аппарат составить а в иванов разработать в соответствие с требование концепция новый умк по отечественный история и историко культурный стандарт на учебник получить положительный заключение по результат научный историко культурный заключение российский исторический общество рио э 001 от 22 04 2015 педагогический заключение российский академия образование 498 от 20 02 2015 и общественный заключение российский книжный союз 1116 от 24 04 2015 экспертиза научно редакционный совет а в торкунов председатель академик ран ректор мгимо н м арсентьев член корреспондент ран сопредседатель научный совет ран по российский 

In [22]:
# Подсчет количества вхождений слова "он"
count_on = sum(1 for text in processed_corpus_texts if 'он__NPRO' in text.split())
print("Количество вхождений слова 'он' в Деткорпусе:", count_on)

Количество вхождений слова 'он' в Деткорпусе: 0


In [23]:
# Подсчет количества вхождений слова "он" без учета части речи
count_on = sum(1 for text in processed_corpus_texts if 'он' in text.split())
print("Количество вхождений слова 'он' в Деткорпусе (без учета части речи):", count_on)

Количество вхождений слова 'он' в Деткорпусе (без учета части речи): 59
